In [1]:
from io import StringIO
import subprocess
import pandas as pd


def run_command(command):
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    return result.stdout

In [2]:
# TODO: run for all GDPR Regions
def get_available_models(region='swedencentral'):
    cmd = f"az cognitiveservices model list -l {region}"
    cmd_output = run_command(cmd)
    full_df = pd.read_json(StringIO(cmd_output))
    full_df["region"] = region

    # parse json in column model into separate columns, but keep the rest of the columns
    model_df = pd.json_normalize(full_df['model'])
    # append model. to column names
    model_df.columns = ['model.' + col for col in model_df.columns]

    # combine the two dataframes
    full_df = pd.concat([full_df.drop(columns=['model']), model_df], axis=1)

    sku_df = full_df.explode('model.skus')
    sku_norm_df = pd.json_normalize(sku_df['model.skus'])
    # append sku. to column names
    sku_norm_df.columns = ['model.sku.' + col for col in sku_norm_df.columns]

    sku_df = sku_df.reset_index(drop=True)
    full_df = pd.concat([sku_df.drop(columns=['model.skus']), sku_norm_df], axis=1)
    return full_df

full_df = get_available_models('swedencentral')
eastus2_df = get_available_models('eastus2')

full_df = pd.concat([full_df, eastus2_df])

In [3]:
# filter deprecated models (model.deprecation.inference lower then today)
full_df['model.deprecation.inference'] = pd.to_datetime(full_df['model.deprecation.inference'], errors='coerce')
current_time = pd.Timestamp.now(tz='UTC')  # Ensure the timestamp is timezone-aware
df = full_df[full_df['model.deprecation.inference'].isna() | (full_df['model.deprecation.inference'] > current_time)]

In [4]:
sc_usable_models_df = df[(~df["model.sku.name"].str.contains(pat="global", case=False, na=True)) & (df["region"] == "swedencentral")]
sc_usable_models_df["model.name"].unique()

array(['dall-e-3', 'whisper', 'gpt-35-turbo', 'gpt-35-turbo-instruct',
       'gpt-35-turbo-16k', 'gpt-4', 'gpt-4-32k', 'gpt-4o', 'gpt-4o-mini',
       'gpt-4o-audio-preview', 'o1', 'o3-mini', 'text-embedding-ada-002',
       'text-embedding-3-large'], dtype=object)

In [5]:

unusable_models_df = df[~df["model.name"].isin(sc_usable_models_df["model.name"])]
unusable_models_df["model.name"].unique()

array(['gpt-4o-mini-realtime-preview', 'gpt-4o-realtime-preview',
       'gpt-4.5-preview', 'gpt-4.1', 'gpt-4.1-nano', 'o1-mini',
       'text-embedding-3-small', 'AI21-Jamba-1.5-Large',
       'AI21-Jamba-1.5-Mini', 'AI21-Jamba-Instruct', 'Codestral-2501',
       'Cohere-command-r', 'Cohere-command-r-08-2024',
       'Cohere-command-r-plus', 'Cohere-command-r-plus-08-2024',
       'Cohere-embed-v3-english', 'Cohere-embed-v3-multilingual',
       'DeepSeek-R1', 'DeepSeek-V3', 'DeepSeek-V3-0324', 'jais-30b-chat',
       'Llama-3.2-11B-Vision-Instruct', 'Llama-3.2-90B-Vision-Instruct',
       'Llama-3.3-70B-Instruct', 'Meta-Llama-3-70B-Instruct',
       'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3.1-405B-Instruct',
       'Meta-Llama-3.1-70B-Instruct', 'Meta-Llama-3.1-8B-Instruct',
       'Ministral-3B', 'Mistral-large', 'Mistral-large-2407',
       'Mistral-Large-2411', 'Mistral-Nemo', 'Mistral-small',
       'mistral-small-2503', 'Phi-3-medium-128k-instruct',
       'Phi-3-medium-4k-instr